In [27]:
! pip install Pinecone

In [28]:
api_key ="pcsk_5jiMcY_M9GJQAnLWv7HQrSgxpkqCeUXtPfrmhBtiNd9vyEeSUngPcdN5oA9mDbWPLrvrhd"

In [29]:
from langchain_community.retrievers import PineconeHybridSearchRetriever


In [30]:
import os 
from pinecone import Pinecone,ServerlessSpec
index_name="hybrid-search-langchain-pinecone"

# Intialize the Pinecone client
pc=Pinecone(api_key=api_key)

# Create the index

if index_name not in pc.list_indexes().names():
    pc.create_index(name=index_name, dimension=384, metric="dotproduct",
                    spec=ServerlessSpec(cloud="aws",region="us-east-1"))
     

In [31]:
index=pc.Index(index_name)
index

In [32]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["HF_TOKEN"]=os.getenv("HF_TOKEN")

from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
embeddings

HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [33]:
from pinecone_text.sparse import BM25Encoder
sparse_encoder = BM25Encoder().default()
sparse_encoder

In [34]:
sentences=[
    "In 2023, I visited Paris",
    "In 2022, I visited New York",
    "In 2021, I visited London"
]

##Tfidf values on these sentence 
sparse_encoder.fit(sentences)

##store the values to a json file 
sparse_encoder.dump("bm25_encoder.json")

##load the values from the json file 
sparse_encoder=BM25Encoder().load("bm25_encoder.json")


  0%|          | 0/3 [00:00<?, ?it/s]

100%|██████████| 3/3 [00:00<00:00, 330.39it/s]


In [35]:

retriever=PineconeHybridSearchRetriever(index=index,embeddings=embeddings,sparse_encoder=sparse_encoder)

In [36]:
retriever.add_texts(texts=sentences)

100%|██████████| 1/1 [00:05<00:00,  5.42s/it]


In [37]:
retriever.invoke("what city did i visit recent")

[Document(metadata={'score': 0.272117913}, page_content='In 2021, I visited London'),
 Document(metadata={'score': 0.227792367}, page_content='In 2022, I visited New York'),
 Document(metadata={'score': 0.201514095}, page_content='In 2023, I visited Paris')]